In [231]:
# !pip install reportlab

In [232]:
import requests
import json

In [233]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = 'boreal-ward-437408-p6'
base_url = "https://generativelanguage.googleapis.com"

In [234]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta/tunedModels',
  headers = headers,
)

In [ ]:
import time
import os
import re

# Instruction to be sent before every new chat session
initial_instruction = (
    "You are a medical assistant specialized in lung and chest diseases. "
    "Your goal is to answer questions related to the symptoms, diagnosis, treatment, prevention, and complications "
    "of the following conditions: Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Pneumonia, Pneumothorax, "
    "Consolidation, Edema, Emphysema, Fibrosis, Pleural Thickening, Hernia. "
    "For any other conditions or general health questions or any other prompts, respond with: 'I can only answer questions related to certain lung and chest conditions.' "
    "If the question is about symptoms or involves symptom-based queries, provide a diagnosis or possible explanations within the scope of the listed diseases."
)

def strip_markdown(text):
    """
    Removes markdown bold (**text**) and italics (*text*) from the given text.
    Additionally, replaces bullet point asterisks (*) with hyphens (-) for consistency.
    """
    # Remove **bold**
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    # Remove *italics*
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    # Replace bullet point asterisks with hyphens, preserving indentation
    text = re.sub(r'^(\s*)\*\s+', r'\1- ', text, flags=re.MULTILINE)
    return text

def ask_medical_chatbot(conversation_history, prompt_suffix=""):
    """
    Sends a prompt to the medical chatbot and returns the response.
    """
    # Initialize the parts with the initial instruction
    parts = [{"text": initial_instruction}]
    
    # Append the conversation history
    for exchange in conversation_history:
        parts.append({"text": f"User: {exchange['user']}"})
        if exchange["assistant"]:
            parts.append({"text": f"Assistant: {exchange['assistant']}"})
    
    # Append any additional prompt suffix if provided
    if prompt_suffix:
        parts.append({"text": prompt_suffix})

    payload = {"contents": [{"parts": parts}]}


    response = requests.post(
        url='https://generativelanguage.googleapis.com/v1beta/tunedModels/updatedmedicalconditions-wkza3c7w9fti:generateContent',
        headers=headers,
        json=payload
    )
    
    if response.status_code == 200:
        response_json = response.json()
        try:
            # Extract the assistant's reply
            answer = response_json['candidates'][0]['content']['parts'][-1]['text']
            return answer.strip()
        except (KeyError, IndexError) as e:
            return "Error: Unable to parse the response structure."
    else:
        return f"Error: {response.status_code} - {response.text}"

def get_condition_details(condition, conversation_history):
    """
    Fetches detailed information for a specific condition from the chatbot.
    """
    # Define a prompt to get detailed information about the condition
    prompt = (
        f"Please provide a detailed overview of {condition}, including its description, suggested testing, complications, lifestyle changes, and treatment options. "
        "Format the response with clear headings and bullet points as appropriate."
    )
    
    # Send the prompt to the chatbot
    condition_detail = ask_medical_chatbot(conversation_history, prompt_suffix=prompt)
    
    # Strip markdown asterisks and normalize bullet points
    condition_detail = strip_markdown(condition_detail)
    print(condition_detail)
    
    return condition_detail

def generate_medical_report(conditions, condition_details_list, condition_links):
    """
    Generates the complete medical report as a string.
    """
    detailed_conditions = "\n\n".join(condition_details_list)
    
    # Generate the links text
    condition_links_text = ""
    for condition in conditions:
        links = condition_links.get(condition, [])
        if links:
            formatted_links = "\n".join([f"- {link}" for link in links])
            condition_links_text += f"{condition}:\n{formatted_links}\n\n"
    
    return f"""
Medical Diagnostic Report: Lung and Chest Conditions

Introduction:
Based on your recent lung imaging, our diagnostic tool has evaluated the likelihood of various lung and chest conditions.
The probabilities below indicate the relative chances of each condition based on the observed symptoms and imaging data.

Summary of Findings:
The top four most probable conditions identified are:
1. {conditions[0]}
2. {conditions[1]}
3. {conditions[2]}
4. {conditions[3]}

Detailed Information:

{detailed_conditions}

Additional Resources:
For more information on these conditions, please refer to the links below:

{condition_links_text}

Recommendations:
- Monitoring Symptoms: Keep track of any changes in your symptoms and report them to your healthcare provider promptly.

Disclaimer:
This report provides an analysis based on imaging data and is intended for informational purposes only. It should not replace professional medical consultation, diagnosis, or treatment. Always seek the advice of your physician or another qualified health provider with any questions you may have regarding a medical condition.
"""

def main():
    # Define links for each condition
    condition_links = {
        "Atelectasis": [
            "https://www.mayoclinic.org/diseases-conditions/atelectasis/symptoms-causes/syc-20369684"
        ],
        "Cardiomegaly": [
            "https://www.mayoclinic.org/diseases-conditions/cardiomegaly/symptoms-causes/syc-20370636"
        ],
        "Effusion": [
            "https://www.webmd.com/lung/pleural-effusion-symptoms-causes-treatments"
        ],
        "Infiltration": [
            "https://www.sciencedirect.com/topics/medicine-and-dentistry/pulmonary-infiltrate#:~:text=Pulmonary%20infiltrates%20can%20represent%20pulmonary,Prompt%20appropriate%20treatment%20is%20vital."
        ],
        "Mass": [
            "https://www.mayoclinic.org/diseases-conditions/lung-cancer/symptoms-causes/syc-20374620",
            "https://www.cancer.org/cancer/types/lung-cancer/detection-diagnosis-staging/signs-symptoms.html"
        ],
        "Nodule": [
            "https://www.lung.org/lung-health-diseases/warning-signs-of-lung-disease/nodules",
            "https://my.clevelandclinic.org/health/diseases/14799-pulmonary-nodules"
        ],
        "Pneumonia": [
            "https://www.mayoclinic.org/diseases-conditions/pneumonia/symptoms-causes/syc-20354204",
            "https://www.lung.org/lung-health-diseases/lung-disease-lookup/pneumonia/symptoms-and-diagnosis"
        ],
        "Pneumothorax": [
            "https://www.mayoclinic.org/diseases-conditions/pneumothorax/symptoms-causes/syc-20350367",
            "https://www.lung.org/lung-health-diseases/lung-disease-lookup/pneumothorax/symptoms-diagnosis-treatment"
        ],
        "Consolidation": [
            "https://radiopaedia.org/articles/consolidation-lung",
            "https://www.healthline.com/health/lung-consolidation"
        ],
        "Edema": [
            "https://www.mayoclinic.org/diseases-conditions/pulmonary-edema/symptoms-causes/syc-20377009"
        ],
        "Emphysema": [
            "https://www.physio-pedia.com/Emphysema"
        ],
        "Fibrosis": [
            "https://www.mayoclinic.org/diseases-conditions/pulmonary-fibrosis/symptoms-causes/syc-20353690"
        ],
        "Pleural Thickening": [
            "https://www.mesothelioma.com/asbestos-cancer/pleural-thickening/"
        ],
        "Hernia": [
            "https://www.urmc.rochester.edu/encyclopedia/content.aspx?contenttypeid=22&contentid=lunghernia"
        ]
    }

    # Example conditions for the report; in practice, these can be dynamically determined based on prior conversation or diagnostic data.
    top_conditions = ["Hernia", "Atelectasis", "Effusion", "Fibrosis"]

    # Initialize a list to hold detailed information for each condition
    condition_details_list = []

    conversation_history = []  # Assuming no prior conversation; adjust as needed

    for condition in top_conditions:
        print(f"Fetching details for {condition}...")
        details = get_condition_details(condition, conversation_history)
        condition_details_list.append(details)
        # Optional: Sleep to avoid hitting API rate limits
        time.sleep(1)

    # Generate the medical report as a string
    report = generate_medical_report(top_conditions, condition_details_list, condition_links)

    # Print the report (optional)
    print("Generating Medical Diagnostic Report...\n")
    print(report)

if __name__ == "__main__":
    main()
